In [32]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import os
# import ffmpeg
import matplotlib.pyplot as plt
import librosa
import librosa.display
# from pydub import AudioSegment as AS
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.utils import Sequence
import keras
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import sys
import h5py
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Input,Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, Conv1D, MaxPooling1D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Bidirectional, LSTM, Reshape
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import pickle
from keras.models import Sequential
from keras.layers import Input,Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, Conv1D, MaxPooling1D
import joblib

In [28]:
BATCH_SIZE = 256

In [29]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))
physical_devices = tf.config.list_physical_devices('GPU')

try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('error')
    
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

cuda_malloc_async
Num GPUs Available:  1


In [30]:
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [33]:
ohe = joblib.load('librosa_all_samples_ohe.joblib')
# decoded_labels= le.inverse_transform(y)

In [34]:
all_samples_train_df = pd.read_pickle('librosa_all_samples_train.pkl')

In [35]:
X_train = np.stack([np.array(val) for val in all_samples_train_df['features'].values], axis=0)
X_train.shape
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std

y_train = np.stack([np.array(val) for val in all_samples_train_df['label'].values], axis=0)
# X_train = np.swapaxes(X_train,1,2)
del all_samples_train_df

train_gen = DataGenerator(X_train, y_train, BATCH_SIZE)

In [36]:
X_train.shape

(9573, 128, 308, 1)

In [23]:
y_train.shape[1]

7

In [12]:
all_samples_valid_df = pd.read_pickle('librosa_all_samples_valid.pkl')
X_valid = np.stack([np.array(val) for val in all_samples_valid_df['features'].values], axis=0)
y_valid = np.stack([np.array(val) for val in all_samples_valid_df['label'].values], axis=0)
X_valid = (X_valid - mean)/std
# X_valid = np.swapaxes(X_valid,1,2)
del all_samples_valid_df

valid_gen = DataGenerator(X_valid, y_valid, BATCH_SIZE)

In [13]:
rlrop = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=4, min_lr=0.000001)
checkpoint_path = 'E:/Dissertation/best_weights.hdf5'
# mcp_save = ModelCheckpoint(checkpoint_path, save_best_only=True,
#                            monitor='val_accuracy',
#                            mode='max')

In [14]:
def build_model1(nr_labels):
    model = Sequential()
    model.add(Input(shape=(308, 128)))
    model.add(Conv1D(256, kernel_size=3, padding='same', activation='relu'))
    model.add(Conv1D(256, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(LSTM(128, return_sequences=True, dropout=0.1))
    model.add(LSTM(128, return_sequences=False, dropout=0.1))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(nr_labels, activation='softmax'))
    return model


In [ ]:
def build_model2(nr_labels):
    model = Sequential()
    model.add(LSTM(64, return_sequences = True, input_shape=(308, 128)))
    model.add(LSTM(64))
    model.add(Dense(nr_labels, activation = 'softmax'))
    return model

In [19]:
def build_model3(nr_labels):
    model = Sequential()
    
    model.add(Conv1D(32, kernel_size=5, padding='same',strides = 1, activation='relu', input_shape=(30, 308)))
    model.add(BatchNormalization())
    model.add(AveragePooling1D())
    model.add(Dropout(0.1))
    
    model.add(Conv1D(32, kernel_size=5, padding='same',strides = 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling1D())
    model.add(Dropout(0.1))
    
    model.add(Conv1D(32, kernel_size=5, padding='same',strides = 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling1D())
    model.add(Dropout(0.1))
    
    model.add(Conv1D(32, kernel_size=5, padding='same',strides = 1, activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling1D())
    model.add(Dropout(0.1)) 
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())   
    
    
    model.add(Dense(nr_labels, activation='softmax'))
    return model

In [24]:
def build_model4(nr_labels):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=5, padding="same", activation='relu', input_shape = (30, 308, 1)))
    model.add(BatchNormalization())
    model.add(AveragePooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, kernel_size=5, padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, kernel_size=5, padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D())
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, kernel_size=5, padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D())
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
#     model.add(Dropout(0.2))
    model.add(Dense(nr_labels, activation='softmax'))
    
    return model



In [25]:
model = build_model4(y_train.shape[1])
model.build()
model.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 30, 308, 64)       1664      
                                                                 
 batch_normalization_15 (Bat  (None, 30, 308, 64)      256       
 chNormalization)                                                
                                                                 
 average_pooling2d_12 (Avera  (None, 15, 154, 64)      0         
 gePooling2D)                                                    
                                                                 
 dropout_15 (Dropout)        (None, 15, 154, 64)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 15, 154, 64)       102464    
                                                                 
 batch_normalization_16 (Bat  (None, 15, 154, 64)     

In [26]:
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=200, batch_size = BATCH_SIZE, callbacks=[rlrop]
)
print('Training done...moving to next set')

Epoch 1/200
225/225 [==============================] - 42s 168ms/step - loss: 1.4055 - accuracy: 0.4815 - val_loss: 1.6818 - val_accuracy: 0.3517 - lr: 0.0010
Epoch 2/200
225/225 [==============================] - 36s 160ms/step - loss: 1.0770 - accuracy: 0.5950 - val_loss: 1.3255 - val_accuracy: 0.4996 - lr: 0.0010
Epoch 3/200
225/225 [==============================] - 36s 159ms/step - loss: 0.9610 - accuracy: 0.6348 - val_loss: 1.0336 - val_accuracy: 0.6282 - lr: 0.0010
Epoch 4/200
225/225 [==============================] - 37s 163ms/step - loss: 0.8781 - accuracy: 0.6673 - val_loss: 0.8838 - val_accuracy: 0.6600 - lr: 0.0010
Epoch 5/200
225/225 [==============================] - 36s 162ms/step - loss: 0.8165 - accuracy: 0.6893 - val_loss: 0.9461 - val_accuracy: 0.6475 - lr: 0.0010
Epoch 6/200
225/225 [==============================] - 36s 160ms/step - loss: 0.7491 - accuracy: 0.7163 - val_loss: 0.8512 - val_accuracy: 0.6817 - lr: 0.0010
Epoch 7/200
225/225 [=========================

KeyboardInterrupt: 

In [ ]:
def plot_performance(history):
    # plot loss
    plt.subplot(211)
    plt.title('Sparse Categorical Crossentropy')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['categorical_accuracy'], color='blue', label='train')
    plt.plot(history.history['val_categorical_accuracy'], color='orange', label='test')

    # save plot to file
    filename = os.getcwd()
    plt.savefig(filename + '/model_performance_plot.png')

In [ ]:
plot_performance(history)

In [ ]:
# all_samples_test_df = pd.read_pickle('all_samples_test.pkl')